In [122]:
import pandas as pd

from pandasql import sqldf

df = pd.read_csv('data.tsv', 
                   sep='\t', 
                   header=None, 
                   names=['query', 'timestamp', 'device'])

#### 1. Диапазон дат, предоставленный для анализа: 
Первый запрос:    31-08-2021 21:00:00 \
Последний запрос: 21-09-2021 20:59:59

In [123]:
min_time_query = """
SELECT MIN(timestamp) as min_time
FROM df
"""

max_time_query = """
SELECT MAX(timestamp) as max_time
FROM df
"""

min_time_unix = sqldf(min_time_query, locals())['min_time'][0]
max_time_unix = sqldf(max_time_query, locals())['max_time'][0]

max_time = pd.to_datetime(max_time_unix, unit='s').strftime('%d-%m-%Y %H:%M:%S')
min_time = pd.to_datetime(min_time_unix, unit='s').strftime('%d-%m-%Y %H:%M:%S')

print(f"Первый запрос:    {min_time}")
print(f"Последний запрос: {max_time}")

Первый запрос:    31-08-2021 21:00:00
Последний запрос: 21-09-2021 20:59:59


#### 2. Количество запросов с текстом "ютуб" в каждой платформе
desktop: 806 \
touch: 732

In [223]:
df['query_lower'] = df['query'].str.lower()

youtube_cnt_query = """
SELECT 
    device, 
    COUNT(*) as query_cnt
FROM df
WHERE query_lower LIKE '%ютуб%'
GROUP BY device
"""
youtube_cnt_table = sqldf(youtube_cnt_query, locals())
print(youtube_cnt_table)


    device  query_cnt
0  desktop        806
1    touch        732


#### 3. Топ10 самых частотных запросов в каждой платформе (desktop, touch).

In [153]:
from tabulate import tabulate

df['query_lower'] = df['query'].str.lower().str.replace("ё", "е")

top_10_touch_query = """
SELECT
    query_lower,
    COUNT(*) as cnt,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM df WHERE device = 'touch'), 2) as freq_of_touch,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM df), 2) as freq_of_all
FROM df
WHERE device = "touch"
GROUP BY query_lower
ORDER BY cnt DESC
LIMIT 10
"""
top_10_touch = sqldf(top_10_touch_query, locals())

total_row_touch = {
    'query_lower': 'ИТОГО:',
    'cnt': top_10_touch['cnt'].sum(),
    'freq_of_touch': top_10_touch['freq_of_touch'].sum(),
    'freq_of_all': top_10_touch['freq_of_all'].sum()
}

separator = pd.DataFrame({col: ['------'] for col in top_10_touch.columns})
 
top_10_touch_with_total = pd.concat([
    top_10_touch,
    separator,
    pd.DataFrame([total_row_touch])
])

In [159]:
top_10_desktop_query = """
SELECT 
    query_lower,
    COUNT(*) as cnt,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM df WHERE device = 'desktop'), 2) as freq_of_desktop,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM df), 2) as freq_of_all
FROM df
WHERE device = "desktop"
GROUP BY query_lower
ORDER BY cnt DESC
LIMIT 10
"""
top_10_desktop = sqldf(top_10_desktop_query, locals())

total_row_desktop = {
    'query_lower': 'ИТОГО:',
    'cnt': top_10_desktop['cnt'].sum(),
    'freq_of_desktop': round(top_10_desktop['freq_of_desktop'].sum(), 2),
    'freq_of_all': round(top_10_desktop['freq_of_all'].sum(), 2)
}

separator = pd.DataFrame({col: ['------'] for col in top_10_desktop.columns})
 
top_10_desktop_with_total = pd.concat([
    top_10_desktop,
    separator,
    pd.DataFrame([total_row_desktop])
])

In [160]:

print(f"touch Топ10 запросов:")
print(tabulate(top_10_touch_with_total, 
               headers=['Запрос', 'Кол-во', '% от touch', '% от всех'], 
               tablefmt='psql', 
               showindex=list(range(1, len(top_10_touch_with_total)-1)) + ["--", ""]))

print(f"desktop Топ10 запросов:")
print(tabulate(top_10_desktop_with_total, 
               headers=['Запрос', 'Кол-во', '% от desktop', '% от всех'], 
               tablefmt='psql', 
               showindex=list(range(1, len(top_10_desktop_with_total)-1)) + ["--", ""]))

touch Топ10 запросов:
+----+-------------------------+----------+--------------+-------------+
|    | Запрос                  | Кол-во   | % от touch   | % от всех   |
|----+-------------------------+----------+--------------+-------------|
| 1  | порно                   | 10076    | 1.27         | 0.84        |
| 2  | секс                    | 8262     | 1.04         | 0.69        |
| 3  | с днем рождения         | 7438     | 0.94         | 0.62        |
| 4  | с днем рождения женщине | 5516     | 0.7          | 0.46        |
| 5  | с днем рождения мужчине | 4339     | 0.55         | 0.36        |
| 6  | хентай                  | 4212     | 0.53         | 0.35        |
| 7  | xxx                     | 3249     | 0.41         | 0.27        |
| 8  | xnxx                    | 2767     | 0.35         | 0.23        |
| 9  | доброе утро             | 2342     | 0.3          | 0.19        |
| 10 | таблица менделеева      | 2278     | 0.29         | 0.19        |
| -- | ------                

Выводы: 1. топ10 запросов на телефон занимают 4.2% от количества всех запросов, на десктоп - 1.23%. Пользователи чаще используют сервис Яндекс Картинки с телефона. 


#### 4. Трафик запросов в течении дня 

Ссылка на дашборд с чартами распределения кол-ва запросов по часам в течение дня в datalens (данные были обработаны, проиллюстированы и прокомментированы 
там): https://datalens.yandex/p59qkl8yi3jmb 

#### 5. Тематики запросов
Выделим основные категории запросов для мобилок и компьютеров: позравления, образование, картинки, календарь, утренние картинки, взрослый контент. Классифицируем запросы по категориям и найдем доли каждой категории относительно запросов на устройство.

In [227]:
import re

categories = {
    'greetings': [
        r'с днем рождения',
        r'поздравл',                         
        r'др'                              
    ],
    'education': [
        r'таблиц',                    
        r'менделеев',                       
        r'английский алфавит',          
        r'алфавит',                        
        r'квадрат',                          
        r'научн'                            
    ],
    'pictures': [
        r'картинк',                         
        r'обои на рабочий стол'                      
    ],
    'calendar': [
        r'календарь',                    
    ],
    'social': [
        r'одноклассник',                 
    ],
    'morning': [
        r'доброе утро'
    ],
    'adult': [
        r'порно',
        r'секс',
        r'xxx',
        r'xnxx',
    ]
}

compiled_categories = {
    category: [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
    for category, patterns in categories.items()
}

def categorize_query(query):
    query = query.lower().strip()
    for category, patterns in compiled_categories.items():
        if any(pattern.search(query) for pattern in patterns):
            return category
    return 'other'

df['category'] = df['query'].apply(categorize_query)

In [228]:
total_touch = df[df['device'] == 'touch'].shape[0]
total_desktop = df[df['device'] == 'desktop'].shape[0]

platform_distribution = (
    df.groupby(['category', 'device'])
    .size()
    .unstack()
)

platform_distribution = platform_distribution.drop('other', axis=0)
#print(platform_distribution)

In [229]:

platform_distribution['touch_part'] = round((platform_distribution['touch'] / total_touch * 100), 2)
platform_distribution['desktop_part'] = round((platform_distribution['desktop'] / total_desktop * 100), 2)

platform_distribution['difference'] = abs((platform_distribution['touch_part'] - platform_distribution['desktop_part']))

contrast_topics = platform_distribution.sort_values('difference', ascending=False)[['touch_part', 'desktop_part', 'difference']]

print(tabulate(contrast_topics, 
               headers=['категория', 'доля (%) на desktop', 'доля (%) на touch', 'разница долей'], 
               tablefmt='psql', 
               showindex=True))

+-------------+-----------------------+---------------------+-----------------+
| категория   |   доля (%) на desktop |   доля (%) на touch |   разница долей |
|-------------+-----------------------+---------------------+-----------------|
| adult       |                  6.34 |                1.49 |            4.85 |
| greetings   |                  4.37 |                2.34 |            2.03 |
| education   |                  1.65 |                3.52 |            1.87 |
| calendar    |                  0.82 |                2.38 |            1.56 |
| pictures    |                  3.72 |                3.02 |            0.7  |
| morning     |                  0.64 |                0.07 |            0.57 |
| social      |                  0.02 |                0.32 |            0.3  |
+-------------+-----------------------+---------------------+-----------------+


Вывод: взрослый контент, поздравления, картинки, картинки "с добрым утром" - это темы, которые более характерны для пользователей мобильных устройств. Образование, календарь, соц.сеть одноклассники - для пользователей компьютеров.